In [1]:
import sys
print(sys.executable)
print(sys.version)
import psycopg2
print(psycopg2.__version__)


/home/asomura/waseda/phish-core/bin/python
3.12.3 (main, Jan  8 2026, 11:30:50) [GCC 13.3.0]
2.9.11 (dt dec pq3 ext lo64)


In [2]:
import socket
socket.gethostname()


'ghpc3'

In [3]:
#!/usr/bin/env python
# coding: utf-8

In [4]:
# === Cell 0 (01_* 用): 新しいRUNを発行してから paths を読む ===
import run_id_registry as runreg
rid = runreg.new_run()  # タイムスタンプで新規 RUN_ID 発行→ファイル & 環境変数に保存

import importlib  # ← ここで初めて paths を import
import _compat.paths as paths
importlib.reload(paths)
print("[01] RUN_ID =", rid, "| paths.RUN_ID =", paths.RUN_ID)


[01] RUN_ID = 2026-02-02_220431 | paths.RUN_ID = 2026-02-02_220431


In [5]:
# === IO PATHS (auto-added guard) ===
import os
from pathlib import Path
from datetime import datetime

if 'RUN_ID' not in globals():
    RUN_ID = os.environ.get("RUN_ID") or datetime.now().strftime("%Y-%m-%d_%H%M%S")
ARTIFACTS = Path("artifacts") / RUN_ID
RAW = ARTIFACTS / "raw"
PROCESSED = ARTIFACTS / "processed"
MODELS = ARTIFACTS / "models"
RESULTS = ARTIFACTS / "results"
HANDOFF = ARTIFACTS / "handoff"
LOGS = ARTIFACTS / "logs"
TRACES = ARTIFACTS / "traces"
for _p in [RAW, PROCESSED, MODELS, RESULTS, HANDOFF, LOGS, TRACES]:
    _p.mkdir(parents=True, exist_ok=True)

# string shortcuts (compat)
RAW_DIR = str(RAW); PROCESSED_DIR = str(PROCESSED); MODELS_DIR = str(MODELS)
RESULTS_DIR = str(RESULTS); HANDOFF_DIR = str(HANDOFF); LOGS_DIR = str(LOGS); TRACES_DIR = str(TRACES)

base_dirs = {'raw': RAW_DIR, 'data': PROCESSED_DIR, 'models': MODELS_DIR,
             'results': RESULTS_DIR, 'handoff': HANDOFF_DIR, 'logs': LOGS_DIR, 'traces': TRACES_DIR}
def resolve(p): p = Path(p); p.mkdir(parents=True, exist_ok=True); return str(p)
def ensure_roots(): pass
def load_config(): return {"root": str(ARTIFACTS), "run_id": RUN_ID}

print(f"✅ IO guard ready -> artifacts/{RUN_ID}")

✅ IO guard ready -> artifacts/2026-02-02_220431


In [6]:
import sys; sys.path.insert(0, "./")
from _compat.paths import resolve, ensure_roots, load_config, compat_base_dirs
ensure_roots()
CFG = load_config()
base_dirs = dict(compat_base_dirs)  # 既存コード互換

In [7]:
# ---- RUN_ID を一度だけ決めて固定（同一ノート／同一実行でブレない）----
import os
from datetime import datetime

if "RUN_ID" not in globals():
    RUN_ID = os.environ.get("RUN_ID") or datetime.now().strftime("%Y-%m-%d_%H%M%S")
    os.environ["RUN_ID"] = RUN_ID  # 同一カーネル内で他ノートにも共有したい場合のみ

# 既存の base_dirs の流儀はそのまま（＝機能不変）
# base_dirs['results'] = f"results/{RUN_ID}"  # artifacts配下に統一するため旧仕様を無効化

# （任意）使うなら解決済みパスを1つだけ作っておく
RESULTS_DIR = resolve(base_dirs['results'])

In [8]:
print("RUN_ID =", RUN_ID)
print("RESULTS_DIR =", RESULTS_DIR)   # 例: .../artifacts/2025-10-12_080143

RUN_ID = 2026-02-02_220431
RESULTS_DIR = artifacts/2026-02-02_220431/results


In [9]:
#!/usr/bin/env python
# coding: utf-8

"""
プログラム名: 01_data_preparation.py
概要: フィッシング検知システムのためのデータ準備（ラベル付きデータの取得）
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-07-20
依存関係: PostgreSQLデータベース（rapids_data）
"""

"""
セル番号: 1
概要: ライブラリのインポートとデータベース接続設定
入力: なし
出力: DB接続設定、基本設定変数
"""

# === CONFIG (追加) ===
import os, json, yaml
from typing import Any, Dict, Optional

cfg: Dict[str, Any] = {}

def _deep_update(base: dict, override: dict) -> dict:
    for k, v in (override or {}).items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            base[k] = _deep_update(base[k], v)
        else:
            base[k] = v
    return base

def load_configuration(config_path: Optional[str] = None,
                       cfg_override: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    01_data_preparation_fixed.py 用のConfig読み込み統合関数
    優先順位: API引数(cfg_override) > 環境変数 > 設定ファイル > デフォルト
    戻り値: 統合済みcfg辞書
    """
    default = {
        "system": {
            "cert_only_mode": False,   # CERT-ONLY廃止（任意）
            "seed": 42,
            "development_mode": False
        },
        "db": {
            "dbname": "rapids_data",
            "user": "postgres",
            "password": "asomura",
            "host": "localhost",
            "port": "5432",
            "read_only": True,
            "timeout_s": 30
        },
        "data": {
            "dev_limit": 1000,
            "balance_data": True,
            "remove_duplicates": True
        }
    }

    cfg_all = dict(default)

    def _load_file(path: str):
        try:
            if path and os.path.exists(path):
                with open(path, "r", encoding="utf-8") as f:
                    if path.endswith((".yml", ".yaml")):
                        return yaml.safe_load(f) or {}
                    return json.load(f) or {}
        except Exception:
            pass
        return {}

    if config_path:
        cfg_all = _deep_update(cfg_all, _load_file(config_path))

    env_map = {
        "CERT_ONLY_MODE": ("system", "cert_only_mode"),
        "DEV_MODE": ("system", "development_mode"),
        "PGDATABASE": ("db", "dbname"),
        "PGUSER": ("db", "user"),
        "PGPASSWORD": ("db", "password"),
        "PGHOST": ("db", "host"),
        "PGPORT": ("db", "port"),
    }
    for env_key, path in env_map.items():
        val = os.getenv(env_key, None)
        if val is None:
            continue
        if isinstance(val, str) and val.lower() in ("true", "false"):
            val = val.lower() == "true"
        node = cfg_all
        for k in path[:-1]:
            node = node.setdefault(k, {})
        node[path[-1]] = val

    if cfg_override:
        cfg_all = _deep_update(cfg_all, cfg_override)

    # CERT-ONLY 強制チェックは削除
    db = cfg_all.get("db", {})
    for key in ["dbname", "user", "host", "port"]:
        if not db.get(key):
            raise ValueError(f"DB設定に必須項目が不足しています: db.{key}")

    global cfg, DB_CONFIG, DEVELOPMENT_MODE, DATA_LIMIT
    cfg = cfg_all
    DB_CONFIG = {
        "dbname": str(cfg_all["db"]["dbname"]),
        "user": str(cfg_all["db"]["user"]),
        "password": str(cfg_all["db"].get("password", "")),
        "host": str(cfg_all["db"]["host"]),
        "port": str(cfg_all["db"]["port"]),
    }
    DEVELOPMENT_MODE = bool(cfg_all["system"].get("development_mode", False))
    DATA_LIMIT = int(cfg_all["data"].get("dev_limit", 1000)) if DEVELOPMENT_MODE else None
    print(f"🔧 設定読み込み完了: DEVELOPMENT_MODE={DEVELOPMENT_MODE}, CERT_ONLY(optional)={cfg_all['system'].get('cert_only_mode', False)}")
    print(f"   DB: {DB_CONFIG['dbname']}@{DB_CONFIG['host']}:{DB_CONFIG['port']} (read_only={cfg_all['db'].get('read_only', True)})")
    return cfg_all

if 'cfg' not in globals() or not cfg:
    try:
        load_configuration(os.getenv("CONFIG_PATH", None))
    except Exception as e:
        print(f"⚠️ Config読み込み警告: {e}")



import os
import sys
import pickle
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime
from pathlib import Path
import json
from typing import Dict, List, Tuple, Optional

# データベース接続設定
DB_CONFIG = {
    'dbname': 'rapids_data',
    'user': 'postgres',
    'password': 'asomura',
    'host': 'localhost',
    'port': '5432'
}

# 開発モードの設定（環境変数で制御）
DEVELOPMENT_MODE = bool(cfg['system'].get('development_mode', False))
if DEVELOPMENT_MODE:
    DATA_LIMIT = int(cfg['data'].get('dev_limit', 1000))
    print(f"⚠️ 警告：開発モードでデータを{DATA_LIMIT}件に制限しています")
    print("⚠️ 本番実行時は開発モードを無効化してください（cfg.system.development_mode=false）")
else:
    DATA_LIMIT = None  # 全データ使用
    print("✅ 本番モード：全データを使用します")
    print("✅ 本番モード：全データを使用します")

# セッションIDの生成
# セッションIDの生成 (RUN_ID に統一)
session_id = RUN_ID
output_dir = Path(RAW_DIR)
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 出力ディレクトリ: {output_dir}")
print(f"🆔 セッションID (RUN_ID): {session_id}")
print("=" * 80)


🔧 設定読み込み完了: DEVELOPMENT_MODE=False, CERT_ONLY(optional)=False
   DB: rapids_data@localhost:5432 (read_only=True)


✅ 本番モード：全データを使用します
✅ 本番モード：全データを使用します
📁 出力ディレクトリ: artifacts/2026-02-02_220431/raw
🆔 セッションID (RUN_ID): 2026-02-02_220431


In [10]:
"""
セル番号: 2
概要: データベースから各テーブルのラベル付きデータを取得
入力: DB_CONFIG, DATA_LIMIT
出力: phishing_data_list, trusted_data
"""

def fetch_labeled_data(conn, limit: Optional[int] = None) -> Tuple[List[pd.DataFrame], pd.DataFrame]:
    """
    ラベル付きデータを取得する関数
    
    Returns:
        Tuple[List[pd.DataFrame], pd.DataFrame]: (フィッシングデータのリスト, 正常データ)
    """
    
    # LIMITの設定
    limit_clause = f"LIMIT {limit}" if limit else ""
    
    # フィッシングデータのクエリ定義
    phishing_queries = {
        'phishtank': f"""
            SELECT 
                cert_domain as domain,
                cert_data,
                'phishtank' as source,
                1 as label
            FROM phishtank_entries
            WHERE cert_status = 'SUCCESS' 
            AND cert_data IS NOT NULL
            AND cert_domain IS NOT NULL
            {limit_clause}
        """,
        'jpcert': f"""
            SELECT 
                domain,
                cert_data,
                'jpcert' as source,
                1 as label
            FROM jpcert_phishing_urls
            WHERE status = 'SUCCESS' 
            AND cert_data IS NOT NULL
            AND domain IS NOT NULL
            {limit_clause}
        """,
        'certificates': f"""
            SELECT 
                domain,
                cert_data,
                'certificates' as source,
                1 as label
            FROM certificates
            WHERE status = 'SUCCESS' 
            AND cert_data IS NOT NULL
            AND domain IS NOT NULL
            {limit_clause}
        """
    }
    
    # 正常データのクエリ
    trusted_query = f"""
        SELECT 
            domain,
            cert_data,
            'trusted' as source,
            0 as label
        FROM trusted_certificates
        WHERE status = 'SUCCESS' 
        AND cert_data IS NOT NULL
        AND domain IS NOT NULL
        {limit_clause}
    """
    
    print("📊 データ取得を開始します...")
    
    # フィッシングデータの取得
    phishing_data_list = []
    
    for source, query in phishing_queries.items():
        print(f"\n🎣 {source.upper()}データを取得中...")
        try:
            df = pd.read_sql(query, conn)
            print(f"  ✅ 取得成功: {len(df):,}件")
            
            if len(df) > 0:
                # データ型の確認と修正
                df['label'] = df['label'].astype(int)
                df['source'] = df['source'].astype(str)
                phishing_data_list.append(df)
            else:
                print(f"  ⚠️ {source}データが見つかりませんでした")
                
        except Exception as e:
            print(f"  ❌ エラー: {str(e)}")
            continue
    
    # 正常データの取得
    print(f"\n✅ 正常データを取得中...")
    try:
        trusted_df = pd.read_sql(trusted_query, conn)
        print(f"  ✅ 取得成功: {len(trusted_df):,}件")
        
        if len(trusted_df) > 0:
            # データ型の確認と修正
            trusted_df['label'] = trusted_df['label'].astype(int)
            trusted_df['source'] = trusted_df['source'].astype(str)
        else:
            print(f"  ⚠️ 正常データが見つかりませんでした")
            trusted_df = pd.DataFrame()
            
    except Exception as e:
        print(f"  ❌ エラー: {str(e)}")
        trusted_df = pd.DataFrame()
    
    return phishing_data_list, trusted_df


# データベース接続とデータ取得
print("\n🔌 データベースに接続中...")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    print("✅ データベース接続成功")
    
    # データ取得
    phishing_data_list, trusted_data = fetch_labeled_data(conn, DATA_LIMIT)
    
    # フィッシングデータの統合
    if phishing_data_list:
        phishing_data = pd.concat(phishing_data_list, ignore_index=True)
        print(f"\n📊 フィッシングデータ統合完了: {len(phishing_data):,}件")
    else:
        phishing_data = pd.DataFrame()
        print("\n⚠️ フィッシングデータが取得できませんでした")
    
    conn.close()
    print("\n✅ データベース接続を閉じました")
    
except Exception as e:
    print(f"❌ データベース接続エラー: {str(e)}")
    phishing_data = pd.DataFrame()
    trusted_data = pd.DataFrame()

# データ取得結果のサマリー
print("\n" + "=" * 80)
print("📊 データ取得結果サマリー")
print("=" * 80)

if not phishing_data.empty:
    print(f"\n🎣 フィッシングデータ:")
    print(f"  - 総件数: {len(phishing_data):,}件")
    print(f"  - ソース別内訳:")
    for source, count in phishing_data['source'].value_counts().items():
        print(f"    - {source}: {count:,}件")
else:
    print("\n⚠️ フィッシングデータなし")

if not trusted_data.empty:
    print(f"\n✅ 正常データ:")
    print(f"  - 総件数: {len(trusted_data):,}件")
else:
    print("\n⚠️ 正常データなし")

print("\n" + "=" * 80)


🔌 データベースに接続中...
✅ データベース接続成功
📊 データ取得を開始します...

🎣 PHISHTANKデータを取得中...


/tmp/ipykernel_258549/2411644574.py:81: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


  ✅ 取得成功: 53,327件

🎣 JPCERTデータを取得中...


/tmp/ipykernel_258549/2411644574.py:81: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


  ✅ 取得成功: 115,866件

🎣 CERTIFICATESデータを取得中...


/tmp/ipykernel_258549/2411644574.py:81: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


  ✅ 取得成功: 196,158件

✅ 正常データを取得中...


/tmp/ipykernel_258549/2411644574.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trusted_df = pd.read_sql(trusted_query, conn)


  ✅ 取得成功: 450,545件

📊 フィッシングデータ統合完了: 365,351件

✅ データベース接続を閉じました

📊 データ取得結果サマリー

🎣 フィッシングデータ:
  - 総件数: 365,351件
  - ソース別内訳:
    - certificates: 196,158件
    - jpcert: 115,866件
    - phishtank: 53,327件

✅ 正常データ:
  - 総件数: 450,545件



In [11]:
"""
セル番号: 3
概要: データの検証、バランス調整、および保存
入力: phishing_data, trusted_data
出力: artifacts/<RUN_ID>/raw/prepared_data.pkl (RAW_DIR配下)
"""

def validate_and_balance_data(phishing_df: pd.DataFrame, trusted_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    データの検証とバランス調整
    
    Args:
        phishing_df: フィッシングデータ
        trusted_df: 正常データ
        
    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: バランス調整後の(フィッシングデータ, 正常データ)
    """
    print("🔍 データ検証とバランス調整を開始...")
    
    # 空のデータフレームチェック
    if phishing_df.empty or trusted_df.empty:
        print("❌ エラー: データが空です")
        return phishing_df, trusted_df
    
    # 必須カラムの確認
    required_columns = ['domain', 'cert_data', 'source', 'label']
    
    for df_name, df in [('phishing', phishing_df), ('trusted', trusted_df)]:
        missing_cols = set(required_columns) - set(df.columns)
        if missing_cols:
            print(f"❌ {df_name}データに必須カラムがありません: {missing_cols}")
            return pd.DataFrame(), pd.DataFrame()
    
    # memoryviewをbytesに変換
    print("\n🔄 cert_data型の変換...")
    phishing_df = phishing_df.copy()
    trusted_df = trusted_df.copy()
    
    phishing_df['cert_data'] = phishing_df['cert_data'].apply(
        lambda x: bytes(x) if isinstance(x, memoryview) else x
    )
    trusted_df['cert_data'] = trusted_df['cert_data'].apply(
        lambda x: bytes(x) if isinstance(x, memoryview) else x
    )
    print("  ✅ memoryviewをbytesに変換完了")
    
    # NULLデータの除去
    print("\n📋 NULL値の処理...")
    
    phishing_before = len(phishing_df)
    trusted_before = len(trusted_df)
    
    phishing_df = phishing_df.dropna(subset=['domain', 'cert_data'])
    trusted_df = trusted_df.dropna(subset=['domain', 'cert_data'])
    
    print(f"  - フィッシング: {phishing_before:,} → {len(phishing_df):,} ({phishing_before - len(phishing_df):,}件削除)")
    print(f"  - 正常: {trusted_before:,} → {len(trusted_df):,} ({trusted_before - len(trusted_df):,}件削除)")
    
    # データソース間の重複除去（優先順位: phishtank > jpcert > certificates）
    print("\n🔄 データソース間の重複処理...")
    
    # 各ソースのデータ数を記録
    source_counts_before = phishing_df['source'].value_counts().to_dict()
    
    # ドメインベースで重複を確認し、優先順位に従って保持
    # ソースの優先順位を定義
    source_priority = {'phishtank': 1, 'jpcert': 2, 'certificates': 3}
    phishing_df['priority'] = phishing_df['source'].map(source_priority)
    
    # 優先順位でソートしてから重複削除（優先度の高いものを残す）
    phishing_df = phishing_df.sort_values('priority').drop_duplicates(subset=['domain'], keep='first')
    phishing_df = phishing_df.drop(columns=['priority'])
    
    # 各ソースの残存データ数を確認
    source_counts_after = phishing_df['source'].value_counts().to_dict()
    
    print("  データソース別の重複削除結果:")
    for source in ['phishtank', 'jpcert', 'certificates']:
        before = source_counts_before.get(source, 0)
        after = source_counts_after.get(source, 0)
        removed = before - after
        print(f"    - {source}: {before:,} → {after:,} ({removed:,}件削除)")
    
    # 同一データフレーム内の重複ドメインの削除
    print("\n🔄 同一データフレーム内の重複ドメイン処理...")
    
    phishing_before = len(phishing_df)
    trusted_before = len(trusted_df)
    
    phishing_df = phishing_df.drop_duplicates(subset=['domain'], keep='first')
    trusted_df = trusted_df.drop_duplicates(subset=['domain'], keep='first')
    
    print(f"  - フィッシング: {phishing_before:,} → {len(phishing_df):,} ({phishing_before - len(phishing_df):,}件削除)")
    print(f"  - 正常: {trusted_before:,} → {len(trusted_df):,} ({trusted_before - len(trusted_df):,}件削除)")
    
    # フィッシングと正常データ間の重複チェック
    print("\n🔍 フィッシングと正常データ間の重複チェック...")
    
    overlapping_domains = set(phishing_df['domain']) & set(trusted_df['domain'])
    if overlapping_domains:
        print(f"  ⚠️ {len(overlapping_domains):,}件の重複ドメインを検出")
        print(f"  → フィッシングデータから削除します")
        phishing_df = phishing_df[~phishing_df['domain'].isin(overlapping_domains)]
    else:
        print("  ✅ フィッシングと正常データ間に重複なし")
    
    # データバランスの調整（少ない方に合わせる）
    print("\n⚖️ データバランスの調整...")
    
    min_count = min(len(phishing_df), len(trusted_df))
    print(f"  - 最小件数: {min_count:,}件")
    
    if len(phishing_df) > min_count:
        # フィッシングデータをサンプリング（ソースごとの比率を維持）
        phishing_df = phishing_df.groupby('source', group_keys=False).apply(
            lambda x: x.sample(n=int(len(x) * min_count / len(phishing_df)), random_state=42)
        )
    
    if len(trusted_df) > min_count:
        # 正常データをサンプリング
        trusted_df = trusted_df.sample(n=min_count, random_state=42)
    
    print(f"  - 調整後フィッシング: {len(phishing_df):,}件")
    print(f"  - 調整後正常: {len(trusted_df):,}件")
    
    return phishing_df, trusted_df


# データの検証とバランス調整
if not phishing_data.empty and not trusted_data.empty:
    phishing_data_balanced, trusted_data_balanced = validate_and_balance_data(phishing_data, trusted_data)
else:
    print("⚠️ データが不足しているため、バランス調整をスキップします")
    phishing_data_balanced = phishing_data
    trusted_data_balanced = trusted_data

# データ保存の準備
print("\n" + "=" * 80)
print("💾 データ保存処理")
print("=" * 80)

# 保存するデータ構造の作成
data_to_save = {
    'phishing_data': phishing_data_balanced,
    'trusted_data': trusted_data_balanced,
    'metadata': {
        'session_id': session_id,
        'created_at': datetime.now().isoformat(),
        'phishing_count': len(phishing_data_balanced),
        'trusted_count': len(trusted_data_balanced),
        'data_sources': ['phishtank', 'jpcert', 'certificates', 'trusted_certificates'],
        'development_mode': DEVELOPMENT_MODE,
        'data_limit': DATA_LIMIT,
        'preprocessing': {
            'null_removed': True,
            'duplicates_removed': True,
            'balanced': True
        }
    }
}

# Pickle形式で保存
output_file = output_dir / 'prepared_data.pkl'
try:
    with open(output_file, 'wb') as f:
        pickle.dump(data_to_save, f)
    print(f"✅ データを保存しました: {output_file}")
    
    # ファイルサイズの確認
    file_size_mb = output_file.stat().st_size / (1024 * 1024)
    print(f"📊 ファイルサイズ: {file_size_mb:.2f} MB")
    
except Exception as e:
    print(f"❌ 保存エラー: {str(e)}")

# 保存データのサマリー表示
if 'data_to_save' in locals():
    print("\n📋 保存データサマリー:")
    print(f"├─ セッションID: {data_to_save['metadata']['session_id']}")
    print(f"├─ フィッシングデータ: {data_to_save['metadata']['phishing_count']:,}件")
    print(f"├─ 正常データ: {data_to_save['metadata']['trusted_count']:,}件")
    print(f"├─ データソース: {', '.join(data_to_save['metadata']['data_sources'])}")
    print(f"└─ 開発モード: {'有効' if data_to_save['metadata']['development_mode'] else '無効'}")

print("\n✨ 01_data_preparation.py の処理が完了しました")


# In[ ]:


🔍 データ検証とバランス調整を開始...

🔄 cert_data型の変換...


  ✅ memoryviewをbytesに変換完了

📋 NULL値の処理...


  - フィッシング: 365,351 → 365,351 (0件削除)
  - 正常: 450,545 → 450,545 (0件削除)

🔄 データソース間の重複処理...
  データソース別の重複削除結果:
    - phishtank: 53,327 → 17,140 (36,187件削除)
    - jpcert: 115,866 → 111,755 (4,111件削除)
    - certificates: 196,158 → 190,075 (6,083件削除)

🔄 同一データフレーム内の重複ドメイン処理...


  - フィッシング: 318,970 → 318,970 (0件削除)
  - 正常: 450,545 → 450,545 (0件削除)

🔍 フィッシングと正常データ間の重複チェック...
  ⚠️ 243件の重複ドメインを検出
  → フィッシングデータから削除します

⚖️ データバランスの調整...
  - 最小件数: 318,727件
  - 調整後フィッシング: 318,727件
  - 調整後正常: 318,727件

💾 データ保存処理


✅ データを保存しました: artifacts/2026-02-02_220431/raw/prepared_data.pkl
📊 ファイルサイズ: 1089.21 MB

📋 保存データサマリー:
├─ セッションID: 2026-02-02_220431
├─ フィッシングデータ: 318,727件
├─ 正常データ: 318,727件
├─ データソース: phishtank, jpcert, certificates, trusted_certificates
└─ 開発モード: 無効

✨ 01_data_preparation.py の処理が完了しました


In [12]:
# === Controller API関数 ===
from typing import Tuple, Dict, Any
import json, hashlib
from datetime import datetime
from pathlib import Path
import pandas as pd

def _safe_bytes(x):
    if isinstance(x, memoryview): return bytes(x)
    return x

def _quick_cert_parse_error_rate(df: pd.DataFrame, sample_n: int = 200) -> float:
    try:
        from cryptography import x509
        from cryptography.hazmat.backends import default_backend
    except Exception:
        return 0.0
    if df.empty: return 0.0
    sub = df.sample(n=min(sample_n, len(df)), random_state=42)
    errors = 0
    total = 0
    for b in sub['cert_data']:
        b = _safe_bytes(b)
        if b is None:
            continue
        total += 1
        try:
            x509.load_der_x509_certificate(b, default_backend())
        except Exception:
            errors += 1
    return (errors / max(total, 1)) if total > 0 else 0.0

def _save_evidence_store(ph_df: pd.DataFrame, tr_df: pd.DataFrame, out_path: Path) -> Path:
    cols = ['domain', 'cert_data', 'source', 'label']
    df = pd.concat([ph_df[cols], tr_df[cols]], ignore_index=True) if (not ph_df.empty and not tr_df.empty) else \
         (ph_df[cols] if not ph_df.empty else tr_df[cols] if not tr_df.empty else pd.DataFrame(columns=cols))
    def _sha256(b):
        b = _safe_bytes(b)
        if not isinstance(b, (bytes, bytearray)): return None
        return hashlib.sha256(b).hexdigest()
    df = df.copy()
    df['cert_hash'] = df['cert_data'].map(_sha256)
    df.drop(columns=['cert_data'], inplace=True)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(out_path, index=False)
    return out_path

def prepare_data(session_id: str, cfg: Dict[str, Any]) -> Tuple[str, Dict[str, str]]:
    try:
        if not isinstance(cfg, dict):
            return "INVALID_INPUT", {"error": "cfgは辞書型で指定してください"}

        load_configuration(cfg_override=cfg)

        out_base = Path(RAW_DIR)  # results → raw に統一
        logs_dir = Path(LOGS_DIR) / "preparation"
        out_base.mkdir(parents=True, exist_ok=True)
        logs_dir.mkdir(parents=True, exist_ok=True)

        import psycopg2
        try:
            conn = psycopg2.connect(**DB_CONFIG, connect_timeout=int(cfg['db'].get('timeout_s', 30)))
        except Exception as e:
            return "ERROR", {"error": f"DB接続に失敗: {e}"}

        try:
            phishing_list, trusted_df = fetch_labeled_data(conn, DATA_LIMIT)
            conn.close()
        except Exception as e:
            return "ERROR", {"error": f"データ取得に失敗: {e}"}

        phishing_df = pd.concat(phishing_list, ignore_index=True) if phishing_list else pd.DataFrame(columns=['domain','cert_data','source','label'])

        total_records = len(phishing_df) + len(trusted_df)
        if total_records < 100:
            return "NOT_FOUND", {"error": f"データ不足: 合計{total_records}件 (<100)",
                                  "phishing": str(len(phishing_df)), "trusted": str(len(trusted_df))}

        parse_err_rate = _quick_cert_parse_error_rate(phishing_df.append(trusted_df, ignore_index=True) if not trusted_df.empty else phishing_df)
        if parse_err_rate > 0.10:
            return "INVALID_INPUT", {"error": f"証明書パース失敗率が高すぎます: {parse_err_rate:.1%} (>10%)"}

        try:
            ph_bal, tr_bal = validate_and_balance_data(phishing_df, trusted_df)
        except Exception as e:
            return "ERROR", {"error": f"データ検証/バランシングに失敗: {e}"}

        prepared_path = out_base / "prepared_data.pkl"
        try:
            data_to_save = {
                'phishing_data': ph_bal,
                'trusted_data': tr_bal,
                'metadata': {
                    'session_id': session_id,
                    'created_at': datetime.now().isoformat(),
                    'phishing_count': len(ph_bal),
                    'trusted_count': len(tr_bal),
                    'data_sources': ['phishtank', 'jpcert', 'certificates', 'trusted_certificates'],
                    'development_mode': DEVELOPMENT_MODE,
                    'data_limit': DATA_LIMIT,
                    'preprocessing': {
                        'null_removed': True,
                        'duplicates_removed': True,
                        'balanced': True
                    }
                }
            }
            with open(prepared_path, "wb") as f:
                import pickle
                pickle.dump(data_to_save, f)
        except Exception as e:
            return "ERROR", {"error": f"prepared_data保存に失敗: {e}"}

        evidence_path = out_base / "evidence_store.parquet"
        try:
            _save_evidence_store(ph_bal, tr_bal, evidence_path)
        except Exception as e:
            return "ERROR", {"error": f"evidence_store保存に失敗: {e}"}

        split_manifest = out_base / "split_manifest.json"
        prov_manifest = out_base / "data_provenance.json"
        try:
            split = {
                "phishing_count": len(ph_bal),
                "trusted_count": len(tr_bal),
                "balanced": True
            }
            split_manifest.write_text(json.dumps(split, ensure_ascii=False, indent=2), encoding="utf-8")

            provenance = {
                "session_id": session_id,
                "created_at": datetime.now().isoformat(),
                "cfg": cfg,
                "source_tables": ["phishtank_entries", "jpcert_phishing_urls", "certificates", "trusted_certificates"]
            }
            prov_manifest.write_text(json.dumps(provenance, ensure_ascii=False, indent=2), encoding="utf-8")
        except Exception as e:
            return "ERROR", {"error": f"メタデータ保存に失敗: {e}"}

        paths = {
            "prepared_data": str(prepared_path),
            "evidence_store": str(evidence_path),
            "split_manifest": str(split_manifest),
            "data_provenance": str(prov_manifest),
            "logs": str(logs_dir / "preparation.log")
        }
        return "OK", paths

    except Exception as e:
        return "ERROR", {"error": f"未処理例外: {e}"}